#Pregunta 3 - 4 en Raya
##Nombre: Jhamil Crespo Rejas
##Carrera: Ingenieria en Ciencias de la Computacion

In [ ]:
import numpy as np

class Board():
    def __init__(self):
        self.state = np.zeros((4,4)) #se modifica la tabla, ahora es de 4x4

    def valid_moves(self):
        return [(i, j) for j in range(4) for i in range(4) if self.state[i, j] == 0] #ahora verifica cuales de las posiciones de la tabla de 4x4 estan vacias

    def update(self, symbol, row, col):
        if self.state[row, col] == 0:
            self.state[row, col] = symbol
        else:
            raise ValueError ("movimiento ilegal !") #marca los movimientos

    def is_game_over(self):
        # comprobar filas y columnas si estan llenas con uno de los simbolos
        if (self.state.sum(axis=0) == 4).sum() >= 1 or (self.state.sum(axis=1) == 4).sum() >= 1: #ahora comprueba si son 4 los espacios que estan llenos
            return 1
        if (self.state.sum(axis=0) == -4).sum() >= 1 or (self.state.sum(axis=1) == -4).sum() >= 1:
            return -1
        # comprobar diagonales si estan llenas con uno de los simbolos
        diag_sums = [
            sum([self.state[i, i] for i in range(4)]),#ahora comprueba si son 4 los espacios que estan llenos
            sum([self.state[i, 4 - i - 1] for i in range(4)]),
        ]
        if diag_sums[0] == 4 or diag_sums[1] == 4:
            return 1
        if diag_sums[0] == -4 or diag_sums[1] == -4:
            return -1
        # empate
        if len(self.valid_moves()) == 0: #si ya no hay espacios vacios en el tablero se acaba el juego
            return 0
        # seguir jugando
        return None

    def reset(self):
        self.state = np.zeros((4,4)) #resetea el juego


In [ ]:
from tqdm import tqdm

class Game():
    def __init__(self, player1, player2):
        player1.symbol = 1 #inicializa los simbolos de los agentes
        player2.symbol = -1
        self.players = [player1, player2]
        self.board = Board() #se inicializa el tablero

    def selfplay(self, rounds=100):
        wins = [0, 0] #inicializa las victorias de ambos jugadores
        for i in tqdm(range(1, rounds + 1)):
            self.board.reset()
            for player in self.players:
                player.reset() #reinicia a los jugadores
            game_over = False
            while not game_over: #se inicia el juego
                for player in self.players:#hace los movimientos para cada jugador
                    action = player.move(self.board)
                    self.board.update(player.symbol, action[0], action[1])
                    for player in self.players:
                        player.update(self.board)#actualiza el tablero
                    if self.board.is_game_over() is not None:
                        game_over = True#acaba el juego
                        break
            self.reward()#cuando acaba el juego asigna las recompensas
            for ix, player in enumerate(self.players):
                if self.board.is_game_over() == player.symbol:
                    wins[ix] += 1
        return wins

    def reward(self):
        winner = self.board.is_game_over()
        if winner == 0: # empate
            for player in self.players:
                player.reward(0.5) #asigna la misma recompensa a ambos agentes
        else: # le damos 1 recompensa al jugador que gana
            for player in self.players:
                if winner == player.symbol:
                    player.reward(1)
                else:
                    player.reward(0)

In [ ]:
class Agent():
    def __init__(self, alpha=0.5, prob_exp=0.5):
        self.value_function = {} # tabla con pares estado -> valor
        self.alpha = alpha         # learning rate
        self.positions = []       # guardamos todas las posiciones de la partida
        self.prob_exp = prob_exp   # probabilidad de explorar

    def reset(self):
        self.positions = []

    def move(self, board, explore=True):
        valid_moves = board.valid_moves()#obtiene los movimientos validos
        # exploracion
        if explore and np.random.uniform(0, 1) < self.prob_exp:#si la exploracion esta habilitada y el numero aleatorio es menor a la probabilidad de exploracion, entonces explora.
            # vamos a una posición aleatoria
            ix = np.random.choice(len(valid_moves))#hace un movimiento aleatorio dentro de los movimientos validos
            return valid_moves[ix]
        # explotacion
        # vamos a la posición con más valor
        max_value = -1000
        for row, col in valid_moves:#recorre todos los movimientos posibles
            next_board = board.state.copy()
            next_board[row, col] = self.symbol
            next_state = str(next_board.reshape(4*4))
            value = 0 if self.value_function.get(next_state) is None else self.value_function.get(next_state)#verifica si los estados siguientes tienen valor, si no es el caso asigna 0
            if value >= max_value:#se queda con el estado que tenga mayor valor
                max_value = value
                best_row, best_col = row, col
        return best_row, best_col#devuelve los indices para el mejor movimiento

    def update(self, board):
        self.positions.append(str(board.state.reshape(4*4)))

    def reward(self, reward):
        # al final de la partida (cuando recibimos la recompensa)
        # iteramos por tods los estados actualizando su valor en la tabla
        for p in reversed(self.positions):#itera empezando del ultimo estado hasta el primero aumentando el valor de estos si es que gano y reduciendo si es que perdio
            if self.value_function.get(p) is None:
                self.value_function[p] = 0
            self.value_function[p] += self.alpha * (reward - self.value_function[p])#asigna  la recompensa en base al coeficiente de aprendizaje
            reward = self.value_function[p]


In [ ]:
agent1 = Agent(prob_exp=0.5)#instancia al agente con una probabilidad de exploracion de 50%
agent2 = Agent()#instancia al segundo agente con la probabilidad de exploracion por defecto (que es igual 50%)

game = Game(agent1, agent2)

game.selfplay(150000)#los agentes juegan entre si 150000 veces

100%|██████████| 150000/150000 [48:53<00:00, 51.14it/s]


[52239, 41296]

In [ ]:
import pandas as pd

pd.set_option('display.max_colwidth', 1000)

funcion_de_valor = sorted(agent1.value_function.items(), key=lambda kv: kv[1], reverse=True)
tabla = pd.DataFrame({'estado': [x[0] for x in funcion_de_valor], 'valor': [x[1] for x in funcion_de_valor]})

tabla#imprimimos todos los resultados de las jugadas: la tabla de estados finales de cada juego y su valor respectivo para el agente 1


,estado,valor
0,[ 0. -1. 0. 1. 0. -1. 0. 1. 0. 0. 0. 1. 0. -1. 0. 1.],1.0
1,[ 0. 0. 0. 1. -1. -1. -1. 1. 0. 0. 0. 1. 0. 0. 0. 1.],1.0
2,[ 0. 0. 0. 1. 0. -1. 0. 1. 0. -1. 0. 1. 0. -1. 0. 1.],1.0
3,[ 0. 0. 0. 1. 0. -1. -1. 1. 0. 0. -1. 1. 0. 0. 0. 1.],1.0
4,[ 0. 0. 0. 1. -1. -1. 0. 1. 0. 0. -1. 1. 0. 0. 0. 1.],1.0
...,...,...
705435,[ 0. 1. 1. 1. -1. 0. 1. -1. 0. 0. 1. 0. -1. 0. -1. -1.],0.0
705436,[ 0. 1. 1. 1. -1. 0. 1. 0. 0. 0. 1. 0. -1. 0. -1. -1.],0.0
705437,[-1. 0. -1. 1. -1. -1. 0. 0. 1. 1. -1. 1. 1. 1. -1. 1.],0.0
705438,[-1. 0. -1. 1. -1. -1. 0. 0. 1. 1. -1. 0. 1. 1. -1. 1.],0.0


In [ ]:
import pickle

with open('agente.pickle', 'wb') as handle:
    pickle.dump(agent1.value_function, handle, protocol=pickle.HIGHEST_PROTOCOL)